First, install and import all neccessary python libraries.

In [1]:
!pip install gensim
!pip install keras
!pip install nltk
!pip install numpy
!pip install pandas
!pip install sklearn
!pip install tensorflow==1.15
!pip install wordcloud

import gensim
from gensim.models import word2vec
from itertools import cycle
import json
from keras.utils import np_utils
import logging
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import LabelEncoder
import sys
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, SimpleRNN, LSTM
from wordcloud import WordCloud, ImageColorGenerator

nltk.download('all')
logging.basicConfig(level=logging.INFO)

Using TensorFlow backend.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Packa

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /Users/lindawong/nltk_data...
[nltk_data]    |   Package unicode_samples i

In [2]:
dataset_folder = input("Enter path of the Yelp dataset folder: ")

Enter path of the Yelp dataset folder: ./yelp_dataset


From the original Yelp dataset, we will extract data from review.json and business.json.

In [3]:
# functions for data extraction
def extract_data(business_data_folder, review_data_folder):
    output_data = {}
    with open(business_data_folder) as bd_handle:
        business = bd_handle.readline()

        while business:
            bd_data = json.loads(business)

            if bd_data['categories']:
                output_data[bd_data['business_id']] = {
                    'categories': [cat.strip() for cat in bd_data['categories'].split(',')],
                    'reviews': []
                }

            business = bd_handle.readline()

        bd_handle.close()

    with open(review_data_folder) as rd_handle:
        review = rd_handle.readline()

        while review:
            rv_data = json.loads(review)

            if rv_data['business_id'] in output_data:
                review = {}
                review['review_id'] = rv_data['review_id']
                review['text'] = rv_data['text']
                review['useful'] = rv_data['useful']
                review['funny'] = rv_data['funny']
                review['cool'] = rv_data['cool']
                output_data[rv_data['business_id']]['reviews'].append(review)

            review = rd_handle.readline()

        bd_handle.close()
        
    return output_data

def write_extracted_data(output_data, output_file):
    data_to_write = []
    for business_id in output_data:
        categories = output_data[business_id]['categories']

        if 'Restaurants' in categories and len(output_data[business_id]['reviews']) > 0:
            cat_to_write = ':'.join(categories)
            for review in output_data[business_id]['reviews']:
                review_id = review['review_id']
                text = review['text']
                useful = str(review['useful'])
                funny = str(review['funny'])
                cool = str(review['cool'])
                data_to_write.append({
                    'business_id': business_id,
                    'categories': cat_to_write,
                    'review_id': review_id,
                    'text': text,
                    'useful': useful,
                    'funny': funny,
                    'cool': cool
                })           

    with open(output_file, 'w') as of_handle:
        json.dump(data_to_write, of_handle)
        of_handle.close()

In [4]:
business_data_folder = os.path.join(dataset_folder, 'business.json')
review_data_folder = os.path.join(dataset_folder, 'review.json')
output_data = extract_data(business_data_folder, review_data_folder)
dataset_file = './dataset.json'
write_extracted_data(output_data, dataset_file)

Now, the data we need from review.json and business.json are stored in the file "dateset.json". Next, we will clean up the data and write the preprocessed datasets to "normalised_data.json".

!!!TODO: explain clipping, making sure dataset has even ranges of labels

In [5]:
# functions for data preprocessing
def preprocess_data(data_file):

    df = pd.read_json(data_file)
    df['useful'] = df['useful'].astype(float)
    df['funny'] = df['funny'].astype(float)
    df['cool'] = df['cool'].astype(float)
    logging.info("Dataset size: {0:d} rows x {1:d} columns".format(df.shape[0], df.shape[1]))

    vote_labels = ['useful', 'funny', 'cool']
    useful_min, useful_max = df['useful'].min(), df['useful'].max()
    funny_min, funny_max = df['funny'].min(), df['funny'].max()
    cool_min, cool_max = df['cool'].min(), df['cool'].max()

    # print out distribution of reviews by their vote counts
    for i in range(len(vote_labels)):
        ranges = np.arange(-1, 100, 10)
        counts = df.groupby(pd.cut(df[vote_labels[i]], ranges)).count()[vote_labels[i]]
        logging.info(counts)
        logging.info("Out of range counts: ", df.shape[0]-counts.sum())

    # save raw vote counts
    df['useful_raw'] = df['useful']
    df['funny_raw'] = df['funny']
    df['cool_raw'] = df['cool']
    # clip vote counts to specific ranges
    df['useful'] = df['useful'].clip(3.0, 27.0)
    df['funny'] = df['funny'].clip(3.0, 20.0)
    df['cool'] = df['cool'].clip(3.0, 24.0)

    # normalize data with min-max scaling
    min_max_scaler = preprocessing.MinMaxScaler()
    useful_scaled = min_max_scaler.fit_transform(np.array(df['useful']).reshape(-1,1))
    df['useful'] = pd.DataFrame(useful_scaled)
    funny_scaled = min_max_scaler.fit_transform(np.array(df['funny']).reshape(-1,1))
    df['funny'] = pd.DataFrame(funny_scaled)
    cool_scaled = min_max_scaler.fit_transform(np.array(df['cool']).reshape(-1,1))
    df['cool'] = pd.DataFrame(cool_scaled)

    ranges = [-0.1, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    processed_data = []

    # balance dataset for each of vote category
    # make sure there is an even number of reviews in each increment of 0.1
    # from range 0.0 to 1.0
    for i in range(len(vote_labels)):
        counts = df.groupby(pd.cut(df[vote_labels[i]], ranges)).count()[vote_labels[i]]
        min_count = counts.min()
        new_df = pd.DataFrame()
        for j in range(10):
            lower = j/10
            upper = (j+1)/10 if j < 9 else 1.1
            new_data = df[(df[vote_labels[i]] >= lower) & (df[vote_labels[i]] < upper)]
            new_data = new_data.sample(frac=1)
            new_data = new_data.head(min_count)
            new_df = new_df.append(new_data)

        processed_data.append(new_df)
    
    return processed_data


def write_extracted_data(useful, funny, cool):
    output_dict = {
        'useful': useful.to_json(orient = 'records'),
        'funny': funny.to_json(orient = 'records'),
        'cool': cool.to_json(orient = 'records')
    }

    with open('normalised_data.json', 'w') as op_handle:
        json.dump(output_dict, op_handle)
        op_handle.close()

In [6]:
processed_data = preprocess_data(dataset_file)
write_extracted_data(processed_data[0], processed_data[1], processed_data[2])

Dataset size: 4201684 rows x 7 columns
useful
(-1, 9]     4148278
(9, 19]       40402
(19, 29]       7225
(29, 39]       2674
(39, 49]       1197
(49, 59]        623
(59, 69]        415
(69, 79]        245
(79, 89]        153
(89, 99]        119
Name: useful, dtype: int64
Out of range counts:  353
funny
(-1, 9]     4180159
(9, 19]       15647
(19, 29]       3281
(29, 39]       1236
(39, 49]        564
(49, 59]        282
(59, 69]        159
(69, 79]         89
(79, 89]         52
(89, 99]         41
Name: funny, dtype: int64
Out of range counts:  174
cool
(-1, 9]     4173089
(9, 19]       20001
(19, 29]       4550
(29, 39]       1778
(39, 49]        913
(49, 59]        477
(59, 69]        316
(69, 79]        179
(79, 89]        134
(89, 99]         95
Name: cool, dtype: int64
Out of range counts:  152


Create a word2vec model using vocabulary of reviews in "dataset.json".

In [7]:
# functions for creating the word2vec model
def read_data(dataset_file):
    logging.info("Reading data...")
    with open(dataset_file, 'r') as fh:
        full_data = json.load(fh)
        useful_data = pd.DataFrame(json.loads(full_data['useful']))
        funny_data = pd.DataFrame(json.loads(full_data['funny']))
        cool_data = pd.DataFrame(json.loads(full_data['cool']))
        all_data = useful_data.append([funny_data, cool_data])
    
    return useful_data, funny_data, cool_data, all_data


def clean_data(tokenized_data_file, all_data, tokenizer, stop_words):
    logging.info("Cleaning data...")
    with open(tokenized_data_file, 'w') as fh:
        reviews = all_data['text'].tolist()

        for index in range(len(reviews)):
            review = reviews[index]
            no_tabs = str(review).replace('\t', ' ').replace('\n', '')
            alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs)
            multi_spaces = re.sub(" +", " ", alphas_only)
            no_spaces = multi_spaces.strip()
            clean_text = no_spaces.lower()
            sentences = tokenizer.tokenize(clean_text)
            sentences = [re.sub("[\.]", "", sentence) for sentence in sentences]

            if len(clean_text) > 0 and clean_text.count(' ') > 0:
                for sentence in sentences:
                    sentence = sentence.split(' ')
                    pruned_sentence = [word for word in sentence if word not in stop_words]
                    sentence = ' '.join(pruned_sentence)
                    if sentence: 
                        fh.write("%s\n" % sentence)

            if (index % 5000) == 0:
                fh.flush()
        fh.close()


def create_wve(tokenized_data_file):
    embedding_size = 250
    min_word_count = 50
    context = 30
    downsampling = 1e-2
    logging.info("Creating word vectors...")
    model = word2vec.Word2Vec(
        word2vec.LineSentence(tokenized_data_file),
        size=embedding_size,
        min_count=min_word_count,
        window=context,
        sample=downsampling)
    model.init_sims(replace=True)
    model.save('./wve.model')
    
    return model

In [8]:
useful_data, funny_data, cool_data, all_data = read_data('normalised_data.json')
tokenized_data_file = './tokennized.txt'
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stop_words = nltk.corpus.stopwords.words()
clean_data(tokenized_data_file, all_data, tokenizer, stop_words)
wve_model = create_wve(tokenized_data_file)

INFO:root:Reading data...
INFO:root:Cleaning data...
INFO:root:Creating word vectors...
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 78521 words, keeping 10014 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 169787 words, keeping 16503 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 262543 words, keeping 21719 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 355778 words, keeping 26363 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 452363 words, keeping 30621 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 548970 words, keeping 34371 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 644108 words, keeping 37909 word types
INFO

INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 15.17% examples, 690505 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 30.07% examples, 693660 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 45.77% examples, 701335 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 52.48% examples, 602933 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 68.89% examples, 632476 words/s, in_qsize 5, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 84.84% examples, 652809 words/s, in_qsize 6, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 2 - PROGRESS: at 95.23% examples, 610412 words/s, in_qsize 5, out_qsize 2
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:work

In [14]:
class BiLSTMModel:
    def __init__(self, data_file, label):
        self.data_file = data_file
        self.label = label
        self.df = []
        self.text = []
        self.num_reviews = 0
        self.max_length = 0
        self.embedding_dim = 250
        self.batch_size = 128
        self.lstm_dim = 100
        self.text_embedding = None
        self.model = None
        self.train_text = None
        self.test_text = None
        self.train_labels = None
        self.test_labels = None

    def read_data(self):        
        # read normalized datatsets
        with open(self.data_file, 'r') as f:
            full_data = json.load(f)
            self.df = pd.DataFrame(json.loads(full_data[self.label]))
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    def create_input_embedding(self):
        # preprocess text
        # remove newlines, spaces, and punctuations
        # lowercase words and separate by commas
        for i in range(self.df.shape[0]):
            line = gensim.utils.simple_preprocess(self.df['text'].iloc[i])
            self.text.append(line)

        length = max([len(s) for s in self.text])
        self.num_reviews = len(self.text)
        
        # convert text to word vector embedding
        text_embedding = np.zeros((self.num_reviews, length, self.embedding_dim))
        for i in range(len(self.text)):
            idx = 0
            for j in range(len(self.text[i])):
                # get jth word of ith review
                word = self.text[i][j]
                # if word is in vocab, add its vector embedding, else skip
                if word in wve_model.wv.vocab:
                    vec = wve_model[word]
                    text_embedding[i,idx,:] = vec
                    idx += 1
            if idx > self.max_length:
                self.max_length = idx
        
        self.text_embedding = text_embedding[:,:self.max_length,:]
        logging.info("Text embedding size: ", text_embedding.shape)
        
    def build_regression_model(self):
        # construct model
        self.model = Sequential()
        self.model.add(Dense(self.batch_size, input_shape=(self.max_length, self.embedding_dim)))
        self.model.add(Dense(300, activation='relu'))
        self.model.add(Dense(150, activation='relu'))
        self.model.add(Bidirectional(LSTM(self.lstm_dim, return_sequences=False)))
        self.model.add(Dense(1, activation='linear'))

        self.model.compile(loss='mae', 
                           optimizer='adam', 
                           metrics=['mae', 'acc'])
        
    def train_regression_model(self):
        labels = self.df[self.label].values
        self.train_text, self.test_text, self.train_labels, self.test_labels = \
            train_test_split(self.text_embedding, labels, test_size=0.3)

        logging.info("Train data size: ", self.train_text.shape, self.train_labels.shape)
        logging.info("Test data size: ", self.test_text.shape, self.test_labels.shape)

        i = self.num_reviews // self.batch_size
        i *= self.batch_size
        self.train_text = self.train_text[:i,:,:]
        self.train_labels = self.train_labels[:i]

        self.model.fit(self.train_text,self.train_labels, 
                  batch_size=self.batch_size, shuffle=True, epochs=5)
        
    def test_regression_model(self):
        self.model.evaluate(self.test_text, self.test_labels)
        
    def build_classification_model(self):
        logging.info("max_length = ", self.max_length)
        # construct model
        self.model = Sequential()
        self.model.add(Dense(self.batch_size, input_shape=(self.max_length, self.embedding_dim)))
        self.model.add(Dense(300, activation='relu'))
        self.model.add(Dense(150, activation='relu'))
        #self.model.add(Bidirectional(LSTM(self.lstm_dim, return_sequences=False)))
        self.model.add(LSTM(self.lstm_dim, return_sequences=False))
        self.model.add(Dense(1, activation='sigmoid'))

        self.model.compile(loss='binary_crossentropy', 
                           #optimizer=tf.keras.optimizers.Adam(0.003),
                           optimizer='adam',
                           metrics=['acc'])
        
    def train_classification_model(self):
        # convert labels to 0 and 1
        # 0 = not useful, 1 = useful
        binary_labels = []
        raw_labels = self.df[self.label].values
        for l in raw_labels:
            if l < 0.5:
                binary_labels.append(0)
            else:
                binary_labels.append(1)
        encoder = LabelEncoder()
        encoder.fit(binary_labels)
        labels = encoder.transform(binary_labels)

        self.train_text, self.test_text, self.train_labels, self.test_labels = \
            train_test_split(self.text_embedding, labels, test_size=0.3)

        logging.info("Train data size: ", self.train_text.shape, self.train_labels.shape)
        logging.info("Test data size: ", self.test_text.shape, self.test_labels.shape)

        i = self.num_reviews // self.batch_size
        i *= self.batch_size
        self.train_text = self.train_text[:i,:,:]
        self.train_labels = self.train_labels[:i]

        self.model.fit(self.train_text,self.train_labels, 
                  batch_size=self.batch_size, shuffle=True, epochs=2)
        
    def test_classification_model(self):
        logging.info("Predict...")
        self.model.evaluate(self.test_text, self.test_labels)
        pred = self.model.predict(self.test_text)
        pred[pred < 0.5] = 0
        pred[pred >= 0.5] = 1
        
        logging.info(classification_report(self.test_labels, pred))        
        
        

In [10]:
BiLSTM_reg_model = BiLSTMModel('normalised_data.json', 'useful')
BiLSTM_reg_model.read_data()
BiLSTM_reg_model.create_input_embedding()
BiLSTM_reg_model.build_regression_model()
BiLSTM_reg_model.train_regression_model()

/Users/lindawong/Documents/Github/yelp_review_prediction/venv/lib/python3.7/site-packages/ipykernel_launcher.py:45: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Text embedding size:  (15190, 939, 250)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train data size:  (10633, 479, 250) (10633,)
Test data size:  (4557, 479, 250) (4557,)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 10633 samples
Epoch 1/5
10633/10633 [==============================] - 468s 44ms/sample - loss: 0.2620 - mean_absolute_error: 0.2620 - acc: 0.1130
Epoch 2/5
10633/10633 [==============================] - 428s 40ms/sample - loss: 0.2534 - mean_absolute_error: 0.2534 - acc: 0.1193
Epoch 3/5
10633/10633 [==============================] - 442s 42ms/sample - loss: 0.2480 - mean_absolute_error: 0.2480 - acc: 0.1243
Epoch 4/5
10633/10633 [==============================] - 383s 36ms/sample - loss: 0.2431 - mean_absolute_error: 0.2431 - acc: 0.1298
Epoch 5/5
10633/10633 [==============================] - 423s 40ms/sample - loss: 0.2418 - mean_absolute_error: 0.2418 - acc: 0.1313


In [11]:
BiLSTM_reg_model.test_regression_model()

4557/4557 [==============================] - 39s 9ms/sample - loss: 0.2457 - mean_absolute_error: 0.2457 - acc: 0.1150


In [15]:
BiLSTM_class_model = BiLSTMModel('normalised_data.json', 'useful')
BiLSTM_class_model.read_data()
BiLSTM_class_model.create_input_embedding()
BiLSTM_class_model.build_classification_model()
BiLSTM_class_model.train_classification_model()

/Users/lindawong/Documents/Github/yelp_review_prediction/venv/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Text embedding size:  (15190, 939, 250)
max_length =  479
Train data size:  (10633, 479, 250) (10633,)
Test data size:  (4557, 479, 250) (4557,)
Train on 10633 samples
Epoch 1/2
10633/10633 [==============================] - 263s 25ms/sample - loss: 0.6934 - acc: 0.4968
Epoch 2/2
10633/10633 [==============================] - 228s 21ms/sample - loss: 0.6933 - acc: 0.4955


In [16]:
BiLSTM_class_model.test_classification_model()

Predict...
4557/4557 [==============================] - 29s 6ms/sample - loss: 0.6932 - acc: 0.4924
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
F1 score:  0.4924292297564187
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2313
           1       0.49      1.00      0.66      2244

    accuracy                           0.49      4557
   macro avg       0.25      0.50      0.33      4557
weighted avg       0.24      0.49      0.32      4557



/Users/lindawong/Documents/Github/yelp_review_prediction/venv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
